In [9]:
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns

%matplotlib inline

PATH = '../data/preprocess/'
PATH_new = '../data/'
file_name = 'third_data2.xlsx'

In [10]:
# 진단일자는 전처리를 위해 넣어준다.
init_columns = ['연구등록번호','Diagnosis','진단일자','Gender','진단시점나이','CMV IgM[Serum]','CMV IgG[Serum]',
'HSV IgM[Serum]','HSV IgG[Serum]','VZV IgM[Serum]','VZV IgG[Serum]','WBC COUNT[Whole blood]','Lymphocyte(#)[Whole blood]',
'Lymphocyte(%)[Whole blood]','Monocyte(#)[Whole blood]','Monocyte(%)[Whole blood]','Neutrophil(#)[Whole blood]',
'Neutrophil(%)[Whole blood]','ESR[Whole blood]','CRP[Serum]']

feature_column = ['Diagnosis','CMV IgM[Serum]','CMV IgG[Serum]',
'HSV IgM[Serum]','HSV IgG[Serum]','VZV IgM[Serum]','VZV IgG[Serum]','WBC COUNT[Whole blood]','Lymphocyte(#)[Whole blood]',
'Lymphocyte(%)[Whole blood]','Monocyte(#)[Whole blood]','Monocyte(%)[Whole blood]','Neutrophil(#)[Whole blood]',
'Neutrophil(%)[Whole blood]','ESR[Whole blood]','CRP[Serum]']

In [11]:
# Read data
df = pd.read_excel(PATH + file_name)
df = df[init_columns]
# Total number
print(df.shape)

# Info
print(df.info())

# Count Diagnosis 0,1,2 
print(df['Diagnosis'].value_counts())

# Diagnosis 기준 count
print(df.groupby('Diagnosis').count())

(8984, 20)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8984 entries, 0 to 8983
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   연구등록번호                      8984 non-null   int64  
 1   Diagnosis                   8984 non-null   int64  
 2   진단일자                        8984 non-null   object 
 3   Gender                      8984 non-null   int64  
 4   진단시점나이                      8984 non-null   int64  
 5   CMV IgM[Serum]              843 non-null    float64
 6   CMV IgG[Serum]              816 non-null    float64
 7   HSV IgM[Serum]              871 non-null    float64
 8   HSV IgG[Serum]              0 non-null      float64
 9   VZV IgM[Serum]              756 non-null    float64
 10  VZV IgG[Serum]              756 non-null    float64
 11  WBC COUNT[Whole blood]      8781 non-null   float64
 12  Lymphocyte(#)[Whole blood]  8791 non-null   float64
 13  Lymphocyte(%)[Whole bl

In [12]:
# change 'VZV IgM[Serum]' and 'Monocyte(#)[Whole blood]' and 'Neutrophil(#)[Whole blood]' into float64
df[['VZV IgM[Serum]','Monocyte(#)[Whole blood]','Neutrophil(#)[Whole blood]']] = df[['VZV IgM[Serum]','Monocyte(#)[Whole blood]','Neutrophil(#)[Whole blood]']].astype(float)

print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8984 entries, 0 to 8983
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   연구등록번호                      8984 non-null   int64  
 1   Diagnosis                   8984 non-null   int64  
 2   진단일자                        8984 non-null   object 
 3   Gender                      8984 non-null   int64  
 4   진단시점나이                      8984 non-null   int64  
 5   CMV IgM[Serum]              843 non-null    float64
 6   CMV IgG[Serum]              816 non-null    float64
 7   HSV IgM[Serum]              871 non-null    float64
 8   HSV IgG[Serum]              0 non-null      float64
 9   VZV IgM[Serum]              756 non-null    float64
 10  VZV IgG[Serum]              756 non-null    float64
 11  WBC COUNT[Whole blood]      8781 non-null   float64
 12  Lymphocyte(#)[Whole blood]  8791 non-null   float64
 13  Lymphocyte(%)[Whole blood]  8791 

## 연구등록번호 기준 결측치 처리

In [13]:
# group by '연구등록번호' and 'Diagnosis'
df_group = df.groupby(['연구등록번호'])
print(len(df_group.groups))


2872


In [14]:
df_fill_group = df_group.apply(lambda x: x.fillna(x.mean()))

/Users/kwontaeyoon/opt/anaconda3/envs/DL/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


In [15]:
df_fill_group.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 8984 entries, (104150, 0) to (10719371, 8983)
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   연구등록번호                      8984 non-null   int64  
 1   Diagnosis                   8984 non-null   int64  
 2   진단일자                        8984 non-null   object 
 3   Gender                      8984 non-null   int64  
 4   진단시점나이                      8984 non-null   int64  
 5   CMV IgM[Serum]              1898 non-null   float64
 6   CMV IgG[Serum]              1854 non-null   float64
 7   HSV IgM[Serum]              2094 non-null   float64
 8   HSV IgG[Serum]              0 non-null      float64
 9   VZV IgM[Serum]              1758 non-null   float64
 10  VZV IgG[Serum]              1850 non-null   float64
 11  WBC COUNT[Whole blood]      8896 non-null   float64
 12  Lymphocyte(#)[Whole blood]  8881 non-null   float64
 13  Lymphocyte(

In [16]:
# df 에 df_fill_group 적용하기
df = df_fill_group
# df.to_excel(PATH + 'third_data_fill.xlsx')

In [17]:
df.info()
print(df.columns)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 8984 entries, (104150, 0) to (10719371, 8983)
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   연구등록번호                      8984 non-null   int64  
 1   Diagnosis                   8984 non-null   int64  
 2   진단일자                        8984 non-null   object 
 3   Gender                      8984 non-null   int64  
 4   진단시점나이                      8984 non-null   int64  
 5   CMV IgM[Serum]              1898 non-null   float64
 6   CMV IgG[Serum]              1854 non-null   float64
 7   HSV IgM[Serum]              2094 non-null   float64
 8   HSV IgG[Serum]              0 non-null      float64
 9   VZV IgM[Serum]              1758 non-null   float64
 10  VZV IgG[Serum]              1850 non-null   float64
 11  WBC COUNT[Whole blood]      8896 non-null   float64
 12  Lymphocyte(#)[Whole blood]  8881 non-null   float64
 13  Lymphocyte(

## Diagnosis 기준 결측치 처리

In [18]:
df_group = df.groupby('Diagnosis')
print(len(df_group.groups))

3


In [19]:
# 각 Group 마다 mean, median, std, min, max 계산
df_group_mean = df_group.mean()
df_group_median = df_group.median()
df_group_std = df_group.std()
df_group_min = df_group.min()
df_group_max = df_group.max()

# Write all into text file
detail_string = '''
--------------------Mean--------------------
{df_group_mean}
--------------------Median--------------------
{df_group_median}
--------------------Std--------------------
{df_group_std}
--------------------Min--------------------
{df_group_min}
--------------------Max--------------------
{df_group_max}
'''

# Write all into text file
# write_file = open(PATH + 'third_data_detail.txt', 'w')
# write_file.write(detail_string.format(df_group_mean=df_group_mean,
#                                         df_group_median=df_group_median,
#                                         df_group_std=df_group_std,
#                                         df_group_min=df_group_min,
#                                         df_group_max=df_group_max))


In [20]:
# median 값으로 채우기
df_fill_group2 = df_group.apply(lambda x: x.fillna(x.median()))
print(df_fill_group2.info())
# Diagnosis 기준 count
df_diagnosis_count = df_fill_group2.groupby('Diagnosis').count()
print(df_diagnosis_count)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 8984 entries, (104150, 0) to (10719371, 8983)
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   연구등록번호                      8984 non-null   int64  
 1   Diagnosis                   8984 non-null   int64  
 2   진단일자                        8984 non-null   object 
 3   Gender                      8984 non-null   int64  
 4   진단시점나이                      8984 non-null   int64  
 5   CMV IgM[Serum]              8984 non-null   float64
 6   CMV IgG[Serum]              8984 non-null   float64
 7   HSV IgM[Serum]              8984 non-null   float64
 8   HSV IgG[Serum]              0 non-null      float64
 9   VZV IgM[Serum]              8984 non-null   float64
 10  VZV IgG[Serum]              8984 non-null   float64
 11  WBC COUNT[Whole blood]      8984 non-null   float64
 12  Lymphocyte(#)[Whole blood]  8984 non-null   float64
 13  Lymphocyte(

/Users/kwontaeyoon/opt/anaconda3/envs/DL/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


In [21]:
# 'CRP[Serum]' 에서 NaN 값을 0으로 채우기
df_fill_group2['CRP[Serum]'] = df_fill_group2['CRP[Serum]'].fillna(0)

In [22]:
print(df_fill_group2.info())

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 8984 entries, (104150, 0) to (10719371, 8983)
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   연구등록번호                      8984 non-null   int64  
 1   Diagnosis                   8984 non-null   int64  
 2   진단일자                        8984 non-null   object 
 3   Gender                      8984 non-null   int64  
 4   진단시점나이                      8984 non-null   int64  
 5   CMV IgM[Serum]              8984 non-null   float64
 6   CMV IgG[Serum]              8984 non-null   float64
 7   HSV IgM[Serum]              8984 non-null   float64
 8   HSV IgG[Serum]              0 non-null      float64
 9   VZV IgM[Serum]              8984 non-null   float64
 10  VZV IgG[Serum]              8984 non-null   float64
 11  WBC COUNT[Whole blood]      8984 non-null   float64
 12  Lymphocyte(#)[Whole blood]  8984 non-null   float64
 13  Lymphocyte(

In [23]:
df = df_fill_group2
df.to_excel(PATH + 'fourth_data2.xlsx')

## 결측치 분석

In [24]:
# 결측치 확인
print(df.isnull().sum())

연구등록번호                           0
Diagnosis                        0
진단일자                             0
Gender                           0
진단시점나이                           0
CMV IgM[Serum]                   0
CMV IgG[Serum]                   0
HSV IgM[Serum]                   0
HSV IgG[Serum]                8984
VZV IgM[Serum]                   0
VZV IgG[Serum]                   0
WBC COUNT[Whole blood]           0
Lymphocyte(#)[Whole blood]       0
Lymphocyte(%)[Whole blood]       0
Monocyte(#)[Whole blood]         0
Monocyte(%)[Whole blood]         0
Neutrophil(#)[Whole blood]       0
Neutrophil(%)[Whole blood]       0
ESR[Whole blood]                 0
CRP[Serum]                       0
dtype: int64


In [25]:
final_column = ['Diagnosis','Gender','진단시점나이','CMV IgM[Serum]','CMV IgG[Serum]',
'HSV IgM[Serum]','VZV IgM[Serum]','VZV IgG[Serum]','WBC COUNT[Whole blood]','Lymphocyte(#)[Whole blood]',
'Lymphocyte(%)[Whole blood]','Monocyte(#)[Whole blood]','Monocyte(%)[Whole blood]','Neutrophil(#)[Whole blood]',
'Neutrophil(%)[Whole blood]','ESR[Whole blood]','CRP[Serum]']

df_final = df[final_column]

In [57]:
print(df_final.info())
df_final.to_excel(PATH + 'final_data.xlsx')



<class 'pandas.core.frame.DataFrame'>
MultiIndex: 9172 entries, (104150, 0) to (10719371, 9171)
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Diagnosis                   9172 non-null   int64  
 1   Gender                      9172 non-null   int64  
 2   진단시점나이                      9172 non-null   int64  
 3   CMV IgM[Serum]              9172 non-null   float64
 4   CMV IgG[Serum]              9172 non-null   float64
 5   HSV IgM[Serum]              9172 non-null   float64
 6   VZV IgM[Serum]              9172 non-null   float64
 7   VZV IgG[Serum]              9172 non-null   float64
 8   WBC COUNT[Whole blood]      9172 non-null   float64
 9   Lymphocyte(#)[Whole blood]  9172 non-null   float64
 10  Lymphocyte(%)[Whole blood]  9172 non-null   float64
 11  Monocyte(#)[Whole blood]    9172 non-null   float64
 12  Monocyte(%)[Whole blood]    9172 non-null   float64
 13  Neutrophil(

In [59]:
# final data mean, median, std, min, max 저장
df_final_mean = df_final.mean()
df_final_median = df_final.median()
df_final_std = df_final.std()
df_final_min = df_final.min()
df_final_max = df_final.max()

# Write all into text file
detail_string = '''
--------------------Mean--------------------
{df_final_mean}
--------------------Median--------------------
{df_final_median}
--------------------Std--------------------
{df_final_std}
--------------------Min--------------------
{df_final_min}
--------------------Max--------------------
{df_final_max}
'''
write_file = open(PATH + 'final_data_detail.txt', 'w')
write_file.write(detail_string.format(df_final_mean=df_final_mean,
                                      df_final_median=df_final_median,
                                      df_final_std=df_final_std,
                                      df_final_min=df_final_min,
                                      df_final_max=df_final_max))


3581

## 데이터 표준화

In [60]:
normalization_feature = ['진단시점나이','CMV IgM[Serum]','CMV IgG[Serum]','HSV IgM[Serum]','VZV IgM[Serum]','VZV IgG[Serum]','WBC COUNT[Whole blood]','Lymphocyte(#)[Whole blood]','Lymphocyte(%)[Whole blood]','Monocyte(#)[Whole blood]','Monocyte(%)[Whole blood]','Neutrophil(#)[Whole blood]','Neutrophil(%)[Whole blood]','ESR[Whole blood]','CRP[Serum]']

# normalize features
df_normalize = df_final.copy()

df_normalize[normalization_feature] = df_normalize[normalization_feature].apply(lambda x: (x - x.mean()) / (x.std()))

In [61]:
df_normalize.head()

Diagnosis  Gender    진단시점나이  CMV IgM[Serum]  CMV IgG[Serum]  \
연구등록번호                                                                  
104150 0          0       1  0.526106       -0.125714       -0.132718   
       1          0       1  0.526106       -0.125714       -0.132718   
261562 2          1       0  1.583882        0.756131       -1.565007   
       3          1       0  1.583882        0.756131       -1.565007   
       4          1       0  1.583882        0.756131       -1.565007   

          HSV IgM[Serum]  VZV IgM[Serum]  VZV IgG[Serum]  \
연구등록번호                                                     
104150 0       -0.207219       -0.189877       -0.067589   
       1       -0.207219       -0.189877       -0.067589   
261562 2        0.267912        0.922669       -1.294243   
       3        0.267912        0.922669       -1.294243   
       4        0.267912        0.922669       -1.294243   

          WBC COUNT[Whole blood]  Lymphocyte(#)[Whole blood]  \
연구등록번호                                                         
104150 0                0.237740                   -1.114502   
       1               -0.472525                    0.155989   
261562 2                1.187960                    0.796664   
       3                2.658881                   -0.929901   
       4                1.351129                   -0.158919   

          Lymphocyte(%)[Whole blood]  Monocyte(#)[Whole blood]  \
연구등록번호                                                           
104150 0                   -1.548119                 -0.429733   
       1                    0.838458                 -0.233326   
261562 2                   -0.080116                  0.552304   
       3                   -2.080377                  0.866555   
       4                   -1.367838                  0.748711   

          Monocyte(%)[Whole blood]  Neutrophil(#)[Whole blood]  \
연구등록번호                                                           
104150 0                 -0.751144                    0.938459   
       1                  0.235499                   -0.720025   
261562 2                  0.175703                    1.838560   
       3                 -0.960432                    4.600145   
       4                 -0.571754                    2.229242   

          Neutrophil(%)[Whole blood]  ESR[Whole blood]  CRP[Serum]  
연구등록번호                                                              
104150 0                   -0.061935         -0.006667   -0.197637  
       1                   -0.061935         -0.006667   -0.197637  
261562 2                    0.208979          0.378758   -0.622844  
       3                    0.208979          0.378758   -0.622844  
       4                    0.208979          0.378758   -0.622844

In [62]:
df_normalize.to_excel(PATH + 'final_data_normalize.xlsx')